In [ ]:
!apt-get update
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,462 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,122 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main 

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1e1d7d59354118940e44a6ead7c63ce42bcd8be5bf19024e5ab4bedad9055c5b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import os
import numpy as np
import random
import csv
import re

from gensim.models import Word2Vec
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

import math
from pyspark.sql.functions import *

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# 텍스트 전처리 함수 정의
def preprocess_text(text, okt, stop_words):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = okt.pos(text) # 토큰화 후 형태소 단위로 분석, [(단어, 품사)] 리턴
    tokens = [word for word, pos in tokens if pos in ['Josa', 'Noun', 'Verb']] # 명사, 동사, 조사 이외의 단어들은 제거
    tokens = [word for word in tokens if word not in stop_words] # 불용어 제거
    return tokens

# CSV 파일 읽어서 저장 기사 Content 튜플
def read_csv_file(file_path):
    global sc
    global spark
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

    # CSV 파일을 스파크 데이터프레임으로 읽기
    df = spark.read.csv(file_path, header=True, inferSchema=True)

    # 스파크 데이터프레임을 Pandas 데이터프레임으로 변환하여 반환
    return df.toPandas()

# TF-IDF 계산
def calcTFIDF(result_data):
    global spark


    lines=sc.parallelize(result_data)
    map1=lines.flatMap(lambda x: [((x[0],x[1]),1)])

    reduce=map1.reduceByKey(lambda x,y:x+y)
    tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))

    map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
    map4=map3.map(lambda x:(x[0],x[1][2]))
    reduce2=map4.reduceByKey(lambda x,y:x+y)
    idf=reduce2.map(lambda x: (x[0],math.log10(len(result_data)/x[1])))
    rdd=tf.join(idf)
    rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
    rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))

    result = rdd.collect()

    spark.stop()

    return pd.DataFrame(result, columns=["url","word","TF","IDF","TF-IDF"])

# Word2Vec 모델 학습 및 저장 함수 정의
def train_and_save_word2vec_model(processed_titles, model_path, seed=SEED):
    # 디렉터리가 존재하지 않으면 생성
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    model = Word2Vec(sentences=processed_titles, vector_size=100, window=5, min_count=1, workers=4, sg=1, seed=seed)
    model.save(model_path)
    return model

# 저장된 Word2Vec 모델 불러오기 함수 정의
def load_word2vec_model(model_path):
    return Word2Vec.load(model_path)

In [ ]:
def removeStopwords(stop_words, dataframe):
    if dataframe is None:
        return None

    result_data = []

    # 데이터프레임의 인덱스를 사용하여 데이터에 접근
    for idx, row in dataframe.iterrows():
        if "Content" not in row or not row["Content"]:  # 'Content' 열이 존재하지 않거나 값이 비어있는 경우 처리
            continue

        # text = row["Content"].lower().replace("subtitle", "").replace("@@div", "구분자").replace("\n", "구분자").replace("img", "구분자")
        # doc = nlp(text)
        text = row["Content"].lower()
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        tokens = okt.pos(text) # 토큰화 후 형태소 단위로 분석, [(단어, 품사)] 리턴
        tokens = [word for word, pos in tokens if pos in ['Josa', 'Noun', 'Verb']] # 명사, 동사, 조사 이외의 단어들은 제거
        tokens = [word for word in tokens if word not in stop_words] # 불용어 제거
        # tokens = ' '.join(tokens)

        words = []
        for token in tokens:
            if len(token) > 0:
                words.append(token)

        for word in words:
            if "URL" not in row or not row["URL"]:  # 'URL' 열이 존재하지 않거나 값이 비어있는 경우 처리
                continue

            result_data.append((row["URL"], word))

    return result_data

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 39.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from konlpy.tag import Okt

# 한국어 불용어 리스트 정의
stop_words = set([
    '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를',
    '으로', '자', '에', '와', '한', '하다', '에서', '에게', '이다', '위해'
])

# 한국어 형태소 분석기 Okt 객체 생성
okt = Okt()

# CSV 파일 읽어서 저장 기사 Content
csv_file_path = '/content/drive/MyDrive/분시컴/result/joongang_news(test).csv'
data = read_csv_file(csv_file_path)

Index(['URL', 'Title', 'Content'], dtype='object')


In [ ]:
# 불용어 제거
data = removeStopwords(stop_words, data)

불용어 제거 완료


In [ ]:
import time

# TF-IDF 계산
start_time = time.time()
tfidf = calcTFIDF(data)
end_time = time.time()
print(f"총 걸린 시간: {end_time - start_time} 초")

TF-IDF 계산 완료


In [ ]:
print(tfidf)

                                               n_id  word  TF       IDF  \
0       https://www.joongang.co.kr/article/25218807     일   8  2.565985   
1       https://www.joongang.co.kr/article/25218807     년   5  2.680648   
2       https://www.joongang.co.kr/article/25218807    부터   2  2.957967   
3       https://www.joongang.co.kr/article/25218807   지난해   1  2.909849   
4       https://www.joongang.co.kr/article/25218807    된다   1  3.086471   
...                                             ...   ...  ..       ...   
196884  https://www.joongang.co.kr/article/25249940    내용   5  3.412044   
196885  https://www.joongang.co.kr/article/25249940     꿈   5  4.031272   
196886  https://www.joongang.co.kr/article/25249940    드리   5  5.045512   
196887  https://www.joongang.co.kr/article/25249940    업무   5  3.593214   
196888  https://www.joongang.co.kr/article/25249940  왔다라며   5  5.221603   

           TF-IDF  
0       20.527878  
1       13.403242  
2        5.915934  
3        2.909849  